<a href="https://colab.research.google.com/github/Blackman9t/Advanced-Data-Science/blob/master/Programming_Assignment_Classification_ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"


Just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [108]:
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet

--2020-02-21 17:47:35--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet [following]
--2020-02-21 17:47:36--  https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59032 (58K) [application/octet-stream]
Saving to: ‘a2.parquet.6’

a2.parquet.6        100%[===================>]  57.65K  --.-KB/s    in 0.02s   

2020-02-21 17:47:37 (2.30 MB/s) - ‘a2.parquet.6’ saved [59032/59032]



Let's first load spark dependencies in Colab

In [109]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

Next we instantiate a SparkContext object called sc

In [110]:
from pyspark import SparkConf, SparkContext
# Set up spark Configuration Object if none exists, otherwise display error msg.
try:
    conf = SparkConf().setMaster('local').setAppName('app')
    sc = SparkContext(conf=conf)
    print('Spark Session Created Successfully!')
except Exception as e:
    print(e)


Cannot run multiple SparkContexts at once; existing SparkContext(app=app, master=local) created by __init__ at <ipython-input-3-e1d1c3853876>:5 


In [111]:
# Let's see the SparkContext
sc

<SparkContext master=local appName=app>

Next we create a Spark Session to connect to the Spark Context Obj

In [112]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('app').getOrCreate()
spark

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [113]:
df=spark.read.load('a2.parquet')
df.show(5)  # This shows the first 5 rows of df as a table

+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
+-----+-----------+-------------------+-------------------+-------------------+
only showing top 5 rows



In [114]:
df.take(5) # This shows the first five lines of the dataframe wrapped in a Row object.

[Row(CLASS=0, SENSORID=26, X=380.66434005495194, Y=-139.3470983812975, Z=-247.93697521077704),
 Row(CLASS=0, SENSORID=29, X=104.74324299209692, Y=-32.27421440203938, Z=-25.105013725863852),
 Row(CLASS=0, SENSORID=8589934658, X=118.11469236129976, Y=45.916682927433534, Z=-87.97203782706572),
 Row(CLASS=0, SENSORID=34359738398, X=246.55394030642543, Y=-0.6122810693132044, Z=-398.18662513951506),
 Row(CLASS=0, SENSORID=17179869241, X=-190.32584900181487, Y=234.7849657520335, Z=-206.34483804019288)]

Next we add the df as a temp table in our session catalog to be able to use SQL queries on it

In [115]:
df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

In [116]:
spark.catalog.listTables()

[Table(name='df', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

Please create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [0]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['X','Y','Z'], outputCol = 'features')

Let's see the count of each category in the label column.

In [118]:
from pyspark.sql.functions import col
df.groupBy("CLASS") \
    .count() \
    .orderBy(col("CLASS").desc()) \
    .show()

+-----+-----+
|CLASS|count|
+-----+-----+
|    1| 1416|
|    0| 1626|
+-----+-----+



In [119]:
# Let's confirm total rows in our data
df.count()

3042

Please instantiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [120]:
df = df.withColumn('label', col('CLASS'))
df.show(5)

+-----+-----------+-------------------+-------------------+-------------------+-----+
|CLASS|   SENSORID|                  X|                  Y|                  Z|label|
+-----+-----------+-------------------+-------------------+-------------------+-----+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|    0|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|    0|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|    0|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|    0|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|    0|
+-----+-----------+-------------------+-------------------+-------------------+-----+
only showing top 5 rows



In [0]:
df.createOrReplaceTempView("df")

Next let's split the data into Training and Testing sets

In [0]:
splits = df.randomSplit([0.75, 0.25])
df_train = splits[0]
df_test = splits[1]

In [123]:
df_train.show(5)

+-----+--------+-------------------+------------------+-------------------+-----+
|CLASS|SENSORID|                  X|                 Y|                  Z|label|
+-----+--------+-------------------+------------------+-------------------+-----+
|    0|       0|-104.60513286125266| -54.0262751326433| 345.25976930323344|    0|
|    0|       0|  56.54732521345129|-7.980106018032676|  95.05162719436447|    0|
|    0|       1|-122.39060867226797| 46.13548501249578|-45.727305937345506|    0|
|    0|       1| 15.798748332829806|-86.21159407546875|   85.2514617870864|    0|
|    0|       2|-60.287010425683505|18.442246406638773|  88.30025324517945|    0|
+-----+--------+-------------------+------------------+-------------------+-----+
only showing top 5 rows



In [124]:
df_test.show(5)

+-----+--------+-------------------+-------------------+-------------------+-----+
|CLASS|SENSORID|                  X|                  Y|                  Z|label|
+-----+--------+-------------------+-------------------+-------------------+-----+
|    0|       2| -258.1817814050372|   66.8675617595676| 433.50053814326026|    0|
|    0|       3| 137.48274121419482|  -93.9756133320119| -220.9712437858673|    0|
|    0|       8| 222.41256717917048|-128.85606064795286| 113.38920172986577|    0|
|    0|      12|-100.92615080557411| -7.468076698886018|  92.98407815956433|    0|
|    0|      12| -42.61570566672431|-40.091552927743706|-6.7322444416323926|    0|
+-----+--------+-------------------+-------------------+-------------------+-----+
only showing top 5 rows



### Logistic Regression Classifier

First Let's try the Logistic Regression algorithm

In [0]:
from pyspark.ml.classification import LogisticRegression

classifier = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

Let’s train and evaluate…


In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [0]:
logReg_model = pipeline.fit(df_train)

In [0]:
prediction = logReg_model.transform(df_train)

In [129]:
prediction.show(5)

+-----+--------+-------------------+------------------+-------------------+-----+--------------------+--------------------+--------------------+----------+
|CLASS|SENSORID|                  X|                 Y|                  Z|label|            features|       rawPrediction|         probability|prediction|
+-----+--------+-------------------+------------------+-------------------+-----+--------------------+--------------------+--------------------+----------+
|    0|       0|-104.60513286125266| -54.0262751326433| 345.25976930323344|    0|[-104.60513286125...|[0.11955840033344...|[0.52985404688191...|       0.0|
|    0|       0|  56.54732521345129|-7.980106018032676|  95.05162719436447|    0|[56.5473252134512...|[0.11955840033344...|[0.52985404688191...|       0.0|
|    0|       1|-122.39060867226797| 46.13548501249578|-45.727305937345506|    0|[-122.39060867226...|[0.11955840033344...|[0.52985404688191...|       0.0|
|    0|       1| 15.798748332829806|-86.21159407546875|   85.251

In [130]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.5298540468819106

Let's evaluate the test data set so we know if we're under or over fitting

In [131]:
prediction = logReg_model.transform(df_test)
binEval.evaluate(prediction)

0.5480153649167734

No, over fitting so let's try Support Vector Machines

### SVM Classifier

Next we import the SVM Classifier

In [0]:
from pyspark.ml.classification import LinearSVC

# Then we define the classifier
classifier = LinearSVC(maxIter=10, regParam=0.1)

In [0]:
# Let's randomly split the data again
splits = df.randomSplit([0.75, 0.25])
df_train = splits[0]
df_test = splits[1]

Next we train the NB classifier with the training set using the pipeline object we created earlier

In [0]:
pipeline = Pipeline(stages=[vectorAssembler, classifier])
svc_model = pipeline.fit(df_train)

In [0]:
prediction = svc_model.transform(df_train)

In [136]:
prediction.show(5)

+-----+--------+-------------------+------------------+-------------------+-----+--------------------+--------------------+----------+
|CLASS|SENSORID|                  X|                 Y|                  Z|label|            features|       rawPrediction|prediction|
+-----+--------+-------------------+------------------+-------------------+-----+--------------------+--------------------+----------+
|    0|       0|  56.54732521345129|-7.980106018032676|  95.05162719436447|    0|[56.5473252134512...|[0.99764249963728...|       0.0|
|    0|       1|-122.39060867226797| 46.13548501249578|-45.727305937345506|    0|[-122.39060867226...|[0.99764249963728...|       0.0|
|    0|       1| 15.798748332829806|-86.21159407546875|   85.2514617870864|    0|[15.7987483328298...|[0.99764249963728...|       0.0|
|    0|       2| -258.1817814050372|  66.8675617595676| 433.50053814326026|    0|[-258.18178140503...|[0.99764249963728...|       0.0|
|    0|       2|-60.287010425683505|18.442246406638773|

In [137]:
binEval.evaluate(prediction) 

0.539612676056338

Let's evaluate the test data

In [138]:
prediction = svc_model.transform(df_test)
binEval.evaluate(prediction) 

0.5194805194805194

### Decision Tree Classifier

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

In [0]:
# Let's randomly split the data againsplits = df.randomSplit([0.75, 0.25])
df_train = splits[0]
df_test = splits[1]

In [0]:
classifier = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [0]:
# Chain vectorAssembler and dt_classifier in a Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [0]:
# Train model.
dt_model = pipeline.fit(df_train)

In [0]:
# Make predictions.
predictions = dt_model.transform(df_test)

In [145]:
predictions.show(5)

+-----+--------+-------------------+-------------------+------------------+-----+--------------------+-------------+-----------+----------+
|CLASS|SENSORID|                  X|                  Y|                 Z|label|            features|rawPrediction|probability|prediction|
+-----+--------+-------------------+-------------------+------------------+-----+--------------------+-------------+-----------+----------+
|    0|       0|-104.60513286125266|  -54.0262751326433|345.25976930323344|    0|[-104.60513286125...|  [498.0,0.0]|  [1.0,0.0]|       0.0|
|    0|       6|   70.5415741207817| -33.17977745901177| 52.41411079917531|    0|[70.5415741207817...|  [497.0,0.0]|  [1.0,0.0]|       0.0|
|    0|      11| -56.73574214427017|  81.98727523527324|-17.89860240383493|    0|[-56.735742144270...|  [498.0,0.0]|  [1.0,0.0]|       0.0|
|    0|      11|   44.2794452012794|-61.434629776826256|  140.920688562067|    0|[44.2794452012794...|  [497.0,0.0]|  [1.0,0.0]|       0.0|
|    0|      18| -13

In [146]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.00779221


In [147]:
binEval.evaluate(predictions)

0.9922077922077922

Let's try our DT Model on the Test data set

In [148]:
predictions = dt_model.transform(df_test)
binEval.evaluate(predictions)

0.9922077922077922

### We have seen no overfitting in all three models. but Clearly with 99% accuracy on both Test and Training sets, the Decision Tree classifier is best

In [0]:
!rm -Rf a2_m2.json

In [0]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [151]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-02-21 17:48:26--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2020-02-21 17:48:26 (73.2 MB/s) - ‘rklib.py’ saved [2540/2540]



In [0]:
import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('a2_m2.json.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('a2_m2.json', zipf)
zipf.close()

In [0]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [154]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = "sisokels@gmail.com"
token = "qzcPyVtngusv8lKS"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, token, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~X0qVJlTSEeq9CwoGlXh86w","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
